In [1]:
import datajoint as dj
import numpy as np
import time
import pymeshfix

dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config["display.limit"] = 20
    
#schema = dj.schema('microns_ta3p100')
#ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema("microns_pinky")
pinky = dj.create_virtual_module("pinky","microns_pinky")

Connecting celiib@10.28.0.34:3306


In [10]:
pinky.PymeshfixDecimatedExcitatoryStitchedMesh()# & "n_vertices>700000"

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices total number of vertices after pymeshfix cleaning,n_triangles total number of faces after pymeshfix cleaning,vertices vertices after pymeshfix cleaning,triangles faces after pymeshfix cleaning,perc_vertices_remaining number of vertices still present after pymeshfix cleaning,perc_triangles_remaining number of faces still present after pymeshfix cleaning
3,648518346341371119,0.35,223210,446728,=BLOB=,=BLOB=,0.99779,1.00152
3,648518346349386137,0.35,27998,56076,=BLOB=,=BLOB=,0.98842,0.99452
3,648518346349470171,0.35,352926,706364,=BLOB=,=BLOB=,0.91522,0.91810
3,648518346349471156,0.35,257594,515604,=BLOB=,=BLOB=,0.94457,0.94117
3,648518346349471500,0.35,232394,465152,=BLOB=,=BLOB=,0.90327,0.89243
3,648518346349471562,0.35,448173,896942,=BLOB=,=BLOB=,0.99005,0.99361
3,648518346349471565,0.35,762502,1525920,=BLOB=,=BLOB=,0.99868,1.00177
3,648518346349471910,0.35,659301,1320374,=BLOB=,=BLOB=,0.94923,0.94471
3,648518346349472574,0.35,524294,1049448,=BLOB=,=BLOB=,0.94572,0.93933
3,648518346349472601,0.35,514470,1029704,=BLOB=,=BLOB=,0.99875,1.00170


# Checking that pymeshfix can't remove some of the basketballs:

In [11]:
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,vertices=[], triangles=[]):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    
    file_loc = pathlib.Path.cwd() / "temp"
    filename = str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename)#,str(filename),str(file_loc)

In [17]:
"""
things tried: 
True False
False False


"""

import pymeshfix
joincomp = False
remove_smallest_components = False

segment_id = 648518346349495660

#get the vertices and faces from datajoint
# get the newly stitched mesh
# get the original mesh

key = dict(segmentation=3,segment_id = segment_id)


In [18]:

#get the vertices 
start = time.time()
mesh = (pinky.Decimation35ExcitatoryStitchedMesh & key).fetch1()
vertices, triangles = mesh['vertices'], mesh['triangles']
meshfix = pymeshfix.MeshFix(vertices,triangles)
verbose=False
meshfix.repair(verbose,joincomp,remove_smallest_components)

print(f"Total time = {time.time() - start}")

Total time = 318.74643778800964


In [16]:
#print the output
write_Whole_Neuron_Off_file(segment_id,meshfix.v,meshfix.f)

Done writing OFF file


'/notebooks/23b_Pymeshfix_Meshes/temp/648518346349495660'